# Live Tracking System Using Python & Folium
# Yuaan Hussain Raheem

## Description:

**This Jupyter Notebook contains Python code for a trip tracking system. It uses various libraries, including Pandas and Folium, to visualize trip data, identify over-speeding segments, and create an interactive map. The code processes trip data from a CSV file, extracts relevant information, and plots it on a map. Continuous over-speeding segments are highlighted to provide insights into the trip.**

## Importing necessary libraries

In [1]:
import pandas as pd
import folium
from folium.plugins import AntPath

## Load trip data from a CSV file

In [2]:
trip_data = pd.read_csv("final_dataset.csv")

## Pre-processing the Dataset

In [3]:
trip_data["Date Time"] = pd.to_datetime(trip_data["Date Time"])

In [4]:
trip_data = trip_data.sort_values("Date Time", ignore_index = True)

In [5]:
trip_group = trip_data.groupby("Trip")

In [6]:
trip = trip_group.get_group(106)

In [7]:
trip.head()

,Date Time,Status,Speed(Km/h),Latitude,Longitude,Reference,Mileage,Geo_Fencing,route,speed_diff,harsh_breaking,harsh_acceleration,Trip,over_speeding
27065,2023-02-08 09:53:47,Ignition On,0,28.92011,70.62502,"0.00 Km From Zahir Pir Service Area-South, Dr....",311,"{'place_id': 249117431, 'licence': 'Data © Ope...",1,0.0,0,0,106,0
27066,2023-02-08 09:53:52,Normal,0,28.92011,70.62502,"0.00 Km From Zahir Pir Service Area-South, Dr....",311,"{'place_id': 249117431, 'licence': 'Data © Ope...",1,0.0,0,0,106,0
27067,2023-02-08 09:53:57,Normal,0,28.92011,70.62502,"0.00 Km From Zahir Pir Service Area-South, Dr....",311,"{'place_id': 249117431, 'licence': 'Data © Ope...",1,0.0,0,0,106,0
27068,2023-02-08 09:54:02,Normal,0,28.92011,70.62502,"0.00 Km From Zahir Pir Service Area-South, Dr....",311,"{'place_id': 249117431, 'licence': 'Data © Ope...",1,0.0,0,0,106,0
27069,2023-02-08 09:54:07,Normal,0,28.92011,70.62502,"0.00 Km From Zahir Pir Service Area-South, Dr....",311,"{'place_id': 249117431, 'licence': 'Data © Ope...",1,0.0,0,0,106,0


## Extract latitude and longitude data from the trip route

In [8]:
trip_route = trip[["Latitude","Longitude"]]
Latitude = trip_route["Latitude"].values
Longitude = trip_route["Longitude"].values

In [9]:
Longitude

array([70.62502, 70.62502, 70.62502, ..., 68.93644, 68.93644, 68.93644])

## Filter locations where over-speeding occurred

In [10]:
# Over-Speeding

over_speeding_location = trip.loc[trip["over_speeding"] == 1,["Latitude","Longitude"]]

In [11]:
over_speeding_location.shape

(544, 2)

In [12]:
over_speeding_location

,Latitude,Longitude
27197,28.88708,70.58680
27217,28.87344,70.55904
27218,28.87314,70.55843
27219,28.87268,70.55751
27220,28.87222,70.55656
...,...,...
28862,27.69209,68.98846
28863,27.69136,68.98782
28864,27.69038,68.98695
28865,27.68966,68.98631


In [13]:
points = over_speeding_location.index
points

Index([27197, 27217, 27218, 27219, 27220, 27221, 27222, 27223, 27224, 27225,
       ...
       28841, 28842, 28847, 28860, 28861, 28862, 28863, 28864, 28865, 28902],
      dtype='int64', length=544)

## Calculate the time intervals between over-speeding points

In [14]:
distances = [points[i+1] - points[i] for i in range(len(points)-1)]

In [15]:
distances[:10]

[20, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [16]:
# pd.Series(points).diff().dropna().values.tolist()

In [17]:
list(enumerate(distances))[:15]

[(0, 20),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 17),
 (13, 1),
 (14, 23)]

## Filter locations where over-speeding occurred continuously

In [18]:
final_lst = []
lst = []

for ind,i in list(enumerate(distances)):
    if i == 1:
        lst.append(ind)
    else:
        final_lst.append(lst)
        lst = []

In [19]:
#final_lst

In [20]:
final_lst = [i for i in final_lst if len(i) > 5]

In [21]:
final_lst[:3]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [20, 21, 22, 23, 24, 25, 26, 27, 28],
 [46, 47, 48, 49, 50, 51, 52]]

In [22]:
ovs = over_speeding_location.iloc[final_lst[0]]

## Interactive Map Visualization of Trip and Over-Speeding

### Description:

**This code creates an interactive map using Folium to visualize a trip's route and over-speeding segments. It includes the following key elements:**

- Trip route displayed using AntPath with custom popups.

- Start and end markers with informative popups.

- Continuous over-speeding segments highlighted with AntPath and custom popups.

- Multiple map tile layers for customization.

- A Layer Control for managing map layers.

In [23]:
my_Map = folium.Map([Latitude[0],Longitude[0]])

layer1 = folium.GeoJson(
    data = (open("pakistan.geojson","r").read()),
    name="Pakistan")

layer1.add_to(my_Map)

AntPath(trip_route.values, delay = 1000, dash_array = [15,30], color = "black", 
        pulse_color = "yellow", weight = 5, opacity = 0.9, tooltip = "<b>Home to Office</b>",
       popup = folium.Popup("""
       <img src="https://techmazone.com/assets/images/logo-v3.png" width="150"/>
       <center><a href="https://techmazone.com">Techma Zone Website</a></center>
       """
       )).add_to(my_Map)

folium.Marker([Latitude[0],Longitude[0]], popup="Home",
             icon = folium.Icon(color = "green", icon = "glyphicon glyphicon-home")).add_to(my_Map)

folium.Marker([Latitude[-1],Longitude[-1]], popup="""
            <img src="https://techmazone.com/assets/images/logo-v3.png" width="150"/>
           <center><a href="https://techmazone.com">Techma Zone Website</a></center>
            """,
             icon = folium.Icon(color = "blue", icon = "glyphicon glyphicon-briefcase")).add_to(my_Map)

#overspeeding

for i in range(len(final_lst)):
    ovs = over_speeding_location.iloc[final_lst[i]]
    AntPath(ovs.values, delay = 300, dash_array = [15,30], color = "red", 
            pulse_color = "yellow", weight = 7, opacity = 1, tooltip = "<b>Home to Office</b>",
           popup = folium.Popup("""
           <img src="https://techmazone.com/assets/images/logo-v3.png" width="150"/>
           <center><a href="https://techmazone.com">Techma Zone Website</a></center>
           """
           )).add_to(my_Map)

    
# add tile layers
folium.TileLayer('openstreetmap').add_to(my_Map)
folium.TileLayer('stamenterrain', attr="stamenterrain").add_to(my_Map)
folium.TileLayer('stamenwatercolor', attr="stamenwatercolor").add_to(my_Map)
folium.TileLayer('https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png', 
 name='CartoDB.DarkMatter', attr="CartoDB.DarkMatter").add_to(my_Map)
folium.TileLayer('https://tile.openstreetmap.bzh/br/{z}/{x}/{y}.png', 
 name='OpenStreetMap.BZH', attr="OpenStreetMap.BZH").add_to(my_Map)



folium.LayerControl().add_to(my_Map)

my_Map.save("Live_tracking_trip_system.html")